**Data augmentation** involves expanding the dataset to enhance performance.

This technique allows us to modify each input data example in a way that preserves its original label.

For instance, in an image classification task where we need to identify if a "cat" is present, we can apply transformations such as rotation, flipping, blurring, or adjusting the color balance.

Despite these changes, the image will still be labeled "cat."

Similarly, in text-based tasks, we can replace words with synonyms or translate the text to another language and then back to the original language.

For audio tasks, we can amplify or reduce different frequency bands.

The goal is to train the model to become robust against these irrelevant data transformations.

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert X_train.shape == (50000, 32, 32, 3)
assert X_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [ ]:
# Convert the data to TensorFlow Datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Data augmentation function
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.2)
    return image, label

# Preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    return image, label

# Dataset preparation function with augmentation option
def prepare_dataset(ds, batch_size=100, augment_data=False):
    if augment_data:
        ds = ds.map(augment, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    ds = ds.shuffle(buffer_size=10000).batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

# Preprocess the data
train_ds = prepare_dataset(train_ds, augment_data=True)
test_ds = prepare_dataset(test_ds, augment_data=False)

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(256, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(128, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dropout(0.3),  # Dropout layer
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(momentum=0.9, learning_rate=0.01, nesterov=True),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_accuracy", factor=0.2, patience=5, min_lr=0.001
)
early_stopping = EarlyStopping(
    monitor="val_accuracy", patience=15, restore_best_weights=True
)
callbacks = [early_stopping, reduce_lr]

# Train the model
history = model.fit(
    train_ds, epochs=100, validation_data=test_ds, batch_size=100, callbacks=callbacks
)

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.tight_layout()
plt.show()